# Case study for Robot Vaccine: SP

In [ ]:
import os
import pandas as pd
import numpy as np
import datetime
import itertools
import matplotlib.pylab as plt
from matplotlib import rc
rc("text", usetex=True)
rc("font", family="serif")

import run_robot
import prepare_data
from importlib import reload
reload(prepare_data)
reload(run_robot)

In [ ]:
# Define the basic data for the case studies

# Mean incubation time
tinc = 5.2

# Mean inffectious time
tinf = 2.9

# Basic reproduction number
basic_rep = 2.5

# Lockdown level
lock_level = 1.0

# Simulation horizon
# A little more than a year when thevaccine should be here
ndays = 36*14

# Mean time in ICU
time_icu = 7

# Define basic paramters
basic_prm = prepare_data.save_basic_parameters(tinc=tinc, tinf=tinf, rep=basic_rep, min_level=lock_level,
    time_icu=time_icu, alternate=0, window=14, ndays=ndays)

pre_cities_data = {
    "S1": [0.584], 
    "E1": [0.015], 
    "I1": [0.001], 
    "R1": [0.4], 
    "population": [44639899], 
    "icu_capacity": [0.000175], 
    "start_date": [pd.to_datetime("2020-2-1")]
}
cities_data = pd.DataFrame(pre_cities_data, index=["SP"])
cities_data

In [ ]:
# Create a target matrix (max infected level)
ncities, ndays = len(cities_data.index), int(basic_prm["ndays"])
target = 0.8*np.ones((ncities, ndays))
target = prepare_data.save_target(cities_data, target)

# Use a forcedif that releases the cities in the end
force_dif = np.zeros((ncities, ndays))

In [ ]:
# Add information on the time series that estimate the need of ICUs
# We are using the time series adjusted considering that the mean ICU stay is 7 days (which lead to larger ICU capacity).

# Define the time series to estimate ICU demand
ts_sp = np.array([0.0, 1.0, 0.01, 0.0, 0.0, 0.0, 0.004, 0.01, 0.01])

ts_drs = ts_sp*np.ones((len(cities_data), len(ts_sp)))
ts_drs = pd.DataFrame(data=ts_drs, index=cities_data.index, columns=[
    "rho_min", "rho_max", "intercept", "trend", "phi_1", "phi_2", "sigma_omega", "state0", "state_less_1"
])
ts_drs["confidence"] = 0.9
ts_drs["time_icu"] = time_icu
cities_data = pd.concat([cities_data, ts_drs], axis=1)
cities_data

In [ ]:
pd.set_option("display.width", 120)

# Simple function to run a test and save results
def run_a_test(basic_prm, result_file, figure_file, cities_data, M, target, force_dif, pools=None, budget=0, tests_off=np.zeros(0, int), tau=3, test_efficacy=0.8, daily_tests=0, proportional_tests=False, verbosity=1):
    run_robot.prepare_optimization(basic_prm, cities_data, M, target, hammer_data, force_dif, pools, 
        verbosity=verbosity, test_budget=budget, tests_off=tests_off, tau=tau, test_efficacy=test_efficacy, 
        daily_tests=daily_tests, proportional_tests=proportional_tests)
    run_robot.optimize_and_show_results(basic_prm, figure_file, result_file, cities_data, target, verbosity=verbosity)

In [ ]:
# Define mobility matrix.
M = prepare_data.convert_mobility_matrix_and_save(cities_data, max_neighbors=5, drs="data/report_drs_mobility.csv")
hammer_data = prepare_data.save_hammer_data(cities_data, 0, basic_prm["min_level"])
run_robot.find_feasible_hammer(basic_prm, cities_data, M, target, hammer_data, out_file=None, 
    incr_all=True, verbosity=1)
M

## Without vacination 

In [ ]:
# Obs: Set max_doses to 0 in robot_dance.jl before running this cell.

reload(run_robot)

# TODO: Get rid of this "test" stuff
tau, test_efficacy, daily_tests, budget = 3, 0.8, 0, 0
tests_off = np.zeros(0, int)

print("******************** Running vaccine simulation")

# Case 1 Optimal tests
basic_prm["alternate"] = 0.0
result_file = f"results/without_vaccine_profile.csv"
figure_file = f"results/without_vaccine_profile.png"
run_a_test(basic_prm, result_file, figure_file, cities_data, M, target, force_dif, None, budget, tests_off, 
    tau, test_efficacy, daily_tests);

In [ ]:
solution = pd.read_csv(result_file, index_col=[0, 1, 2])

def agregate(solution, labels):
    values = solution.loc[labels[0]].sum()
    for i in range(1, len(labels)):
        values += solution.loc[labels[i]].sum()
    return values

i = agregate(solution, ["i", "ir"])
rt = solution.loc["rt", -1, -1]
run_robot.plot_result("SP", basic_prm, rt, i, hammer_data["duration"], start_date="2020-02-01")

Without vacination the epidemic will last till mid July what it recedes due to herd immunity.

## With vacination blocking transmission

In [ ]:
# Before running this cell define the number of available vaccines
# and set the system to block transmission;
reload(run_robot)

# TODO: Get rid of this "test" stuff
tau, test_efficacy, daily_tests, budget = 3, 0.8, 0, 0
tests_off = np.zeros(0, int)

print("******************** Running vaccine simulation")

# Case 1 Optimal tests
basic_prm["alternate"] = 0.0
result_file = f"results/vaccine_profile.csv"
figure_file = f"results/vaccine_profile.png"
run_a_test(basic_prm, result_file, figure_file, cities_data, M, target, force_dif, None, budget, tests_off, 
    tau, test_efficacy, daily_tests);

In [ ]:
solution = pd.read_csv(result_file, index_col=[0, 1, 2])
i = agregate(solution, ["i", "ir"])
rt = solution.loc["rt", -1, -1]
run_robot.plot_result("SP", basic_prm, rt, i, hammer_data["duration"], start_date="2020-02-01")

With vaccination the epidemic ends by May. The need for social distancing from mid April on is low.

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

bins = [[0, 19], [20, 49], [50, 64], [65, 200]]
colors = ["C0", "C1", "C2", "C3"]

def retrieve_vaccination_profile(sol, pop=0, dose=0):
    sol = sol.loc[:, pop, :]
    seir = sol.loc["s", :] + sol.loc["e", :] + sol.loc["i", :] + sol.loc["r", :]
    applied = sol.loc["v", dose]*seir.loc[dose]
    return applied[14:]

In [ ]:
for i in range(len(bins)):
    retrieve_vaccination_profile(solution, i, 0).plot(lw=3, color=colors[i], alpha=0.7, label=f"{bins[i]}, dose 1")
    retrieve_vaccination_profile(solution, i, 1).plot(lw=3, alpha=0.7, color=colors[i], ls="--", label=f"{bins[i]}, dose 2")
ticks = range(0, solution.loc["s", 0, 0, :].shape[1], 28)
plt.xticks(ticks, ticks)
plt.legend()
plt.title("Vaccination");

In [ ]:
total_dose1 = retrieve_vaccination_profile(solution, 0, 0)
for i in range(1, len(bins)):
    total_dose1 += retrieve_vaccination_profile(solution, i, 0)
total_dose2 = retrieve_vaccination_profile(solution, 0, 1)
for i in range(1, len(bins)):
    total_dose2 += retrieve_vaccination_profile(solution, i, 1)
total_dose1.plot(lw=3, alpha=0.7, label="Total dose 1")
total_dose2.plot(lw=3, ls="--", alpha=0.7, label="Total dose 2")
both = total_dose1 + total_dose2 
both.plot(lw=3, alpha=0.3, label="Both doses")
plt.xticks(ticks, ticks)
plt.legend()
plt.title("Total daily vaccination");
print("Total doses", sum(both))

In this case the second dose is postponed to the 60th day. The first group to be vaccinated is 65+ and the second is 50+. Only 65+ needs to get the second dose before the epidemic recedes and the epidemic is controled. After that the vaccination can go slower. In group that has higher R0, 20+, only a few get vaccines before the epidemic is controled.

## With vacination blocking bad symptoms

In [ ]:
# Before running this cell define the number of available vaccines
# and set the system to block symptoms.
reload(run_robot)

# TODO: Get rid of this "test" stuff
tau, test_efficacy, daily_tests, budget = 3, 0.8, 0, 0
tests_off = np.zeros(0, int)

print("******************** Running vaccine simulation")

# Case 1 Optimal tests
basic_prm["alternate"] = 0.0
result_file = f"results/vaccine_profile.csv"
figure_file = f"results/vaccine_profile.png"
run_a_test(basic_prm, result_file, figure_file, cities_data, M, target, force_dif, None, budget, tests_off, 
    tau, test_efficacy, daily_tests);

In [ ]:
solution = pd.read_csv(result_file, index_col=[0, 1, 2])
i = agregate(solution, ["i", "ir"])
rt = solution.loc["rt", -1, -1]
run_robot.plot_result("SP", basic_prm, rt, i, hammer_data["duration"], start_date="2020-02-01")

The epidemic is also controlled sooner, but using more social distancing (14 more days in moderate) than when the vaccine blocks transmission. 

In [ ]:
for i in range(len(bins)):
    retrieve_vaccination_profile(solution, i, 0).plot(lw=3, color=colors[i], alpha=0.7, label=f"{bins[i]}, dose 1")
    retrieve_vaccination_profile(solution, i, 1).plot(lw=3, alpha=0.7, color=colors[i], ls="--", label=f"{bins[i]}, dose 2")
ticks = range(0, solution.loc["s", 0, 0, :].shape[1], 28)
plt.xticks(ticks, ticks)
plt.legend()
plt.title("Vaccination");

In [ ]:
total_dose1 = retrieve_vaccination_profile(solution, 0, 0)
for i in range(1, len(bins)):
    total_dose1 += retrieve_vaccination_profile(solution, i, 0)
total_dose2 = retrieve_vaccination_profile(solution, 0, 1)
for i in range(1, len(bins)):
    total_dose2 += retrieve_vaccination_profile(solution, i, 1)
total_dose1.plot(lw=3, alpha=0.7, label="Total dose 1")
total_dose2.plot(lw=3, ls="--", alpha=0.7, label="Total dose 2")
both = total_dose1 + total_dose2 
both.plot(lw=3, alpha=0.3, label="Both doses")
plt.xticks(ticks, ticks)
plt.legend()
plt.title("Total daily vaccination");
print("Total doses", sum(both))

Again the second dose is postponed to something close to the 60th day, but now only the two older groups receive doses in the first phase and there is more effor to give both doses to them. The group 20+ only receives its first dose at day 112. 

## With vacination blocking transmission/infection but with 0.5% vaccines available since the start

In [ ]:
# Before running this cell define the number of available vaccines to 0.005 in the first days
# and set the system to block transmission.
reload(run_robot)

# TODO: Get rid of this "test" stuff
tau, test_efficacy, daily_tests, budget = 3, 0.8, 0, 0
tests_off = np.zeros(0, int)

print("******************** Running vaccine simulation")

# Case 1 Optimal tests
basic_prm["alternate"] = 0.0
result_file = f"results/vaccine_profile.csv"
figure_file = f"results/vaccine_profile.png"
run_a_test(basic_prm, result_file, figure_file, cities_data, M, target, force_dif, None, budget, tests_off, 
    tau, test_efficacy, daily_tests);

In [ ]:
solution = pd.read_csv(result_file, index_col=[0, 1, 2])
i = agregate(solution, ["i", "ir"])
rt = solution.loc["rt", -1, -1]
run_robot.plot_result("SP", basic_prm, rt, i, hammer_data["duration"], start_date="2020-02-01")

Now the pandemic is controlled much sooner, by the end of April. 

In [ ]:
for i in range(len(bins)):
    retrieve_vaccination_profile(solution, i, 0).plot(lw=3, color=colors[i], alpha=0.7, label=f"{bins[i]}, dose 1")
    retrieve_vaccination_profile(solution, i, 1).plot(lw=3, alpha=0.7, color=colors[i], ls="--", label=f"{bins[i]}, dose 2")
ticks = range(0, solution.loc["s", 0, 0, :].shape[1], 28)
plt.xticks(ticks, ticks)
plt.legend()
plt.title("Vaccination");

In [ ]:
total_dose1 = retrieve_vaccination_profile(solution, 0, 0)
for i in range(1, len(bins)):
    total_dose1 += retrieve_vaccination_profile(solution, i, 0)
total_dose2 = retrieve_vaccination_profile(solution, 0, 1)
for i in range(1, len(bins)):
    total_dose2 += retrieve_vaccination_profile(solution, i, 1)
total_dose1.plot(lw=3, alpha=0.7, label="Total dose 1")
total_dose2.plot(lw=3, ls="--", alpha=0.7, label="Total dose 2")
both = total_dose1 + total_dose2 
both.plot(lw=3, alpha=0.3, label="Both doses")
plt.xticks(ticks, ticks)
plt.legend()
plt.title("Total daily vaccination");
print("Total doses", sum(both))

In this case the second dose is postponed a little less, and it is used after day 56. Note that here **the young group 20+ gets the very first doses** followed by 50+ and only after that the 65+ group gets its shots. But they are the first to receive the second dose. This indicates indicating that the 20+ higher R0 is relevant and is partially attenuated by these initial doses. 

## With vacination blocking bad symptoms but with 0.5% vaccines available since the start

In [ ]:
# Before running this cell define the number of available vaccines to 0.005 in the first days
# and set the system to block symptoms.
reload(run_robot)

# TODO: Get rid of this "test" stuff
tau, test_efficacy, daily_tests, budget = 3, 0.8, 0, 0
tests_off = np.zeros(0, int)

print("******************** Running vaccine simulation")

# Case 1 Optimal tests
basic_prm["alternate"] = 0.0
result_file = f"results/vaccine_profile.csv"
figure_file = f"results/vaccine_profile.png"
run_a_test(basic_prm, result_file, figure_file, cities_data, M, target, force_dif, None, budget, tests_off, 
    tau, test_efficacy, daily_tests);

In [ ]:
solution = pd.read_csv(result_file, index_col=[0, 1, 2])
i = agregate(solution, ["i", "ir"])
rt = solution.loc["rt", -1, -1]
run_robot.plot_result("SP", basic_prm, rt, i, hammer_data["duration"], start_date="2020-02-01")

Again, it takes a little longer to control the pandemic if the vaccine only attenuates bad symptoms. And now by 42 days!

In [ ]:
for i in range(len(bins)):
    retrieve_vaccination_profile(solution, i, 0).plot(lw=3, color=colors[i], alpha=0.7, label=f"{bins[i]}, dose 1")
    retrieve_vaccination_profile(solution, i, 1).plot(lw=3, alpha=0.7, color=colors[i], ls="--", label=f"{bins[i]}, dose 2")
ticks = range(0, solution.loc["s", 0, 0, :].shape[1], 28)
plt.xticks(ticks, ticks)
plt.legend()
plt.title("Vaccination");

In [ ]:
total_dose1 = retrieve_vaccination_profile(solution, 0, 0)
for i in range(1, len(bins)):
    total_dose1 += retrieve_vaccination_profile(solution, i, 0)
total_dose2 = retrieve_vaccination_profile(solution, 0, 1)
for i in range(1, len(bins)):
    total_dose2 += retrieve_vaccination_profile(solution, i, 1)
total_dose1.plot(lw=3, alpha=0.7, label="Total dose 1")
total_dose2.plot(lw=3, ls="--", alpha=0.7, label="Total dose 2")
both = total_dose1 + total_dose2 
both.plot(lw=3, alpha=0.3, label="Both doses")
plt.xticks(ticks, ticks)
plt.legend()
plt.title("Total daily vaccination");
print("Total doses", sum(both))

Now the second dose start jut after the 28th day, it is almost not posponed. Once again all the focus is on the two older groups. This is natural, as the vaccine can not block the transmission from the 20+ group that has higher R0. 